In [1]:
import trimesh
import potpourri3d as pp3d
import numpy as np
import pyvista as pv 

In [2]:
import trimesh
import numpy as np
from scipy.spatial import KDTree

def unify_vertices(mesh, tolerance=1e-5):
    # Extract vertices and faces from the mesh
    vertices = mesh.vertices
    faces = mesh.faces

    # Step 1: Find unique vertices within the given tolerance
    kdtree = KDTree(vertices)
    unique_indices = {}
    canonical_vertices = []
    
    for i, vertex in enumerate(vertices):
        if i in unique_indices:
            continue  # Skip if already mapped to a canonical vertex
        
        # Find all vertices within the tolerance distance
        duplicate_indices = kdtree.query_ball_point(vertex, tolerance)
        
        # Use the first duplicate as the "canonical" vertex and map all duplicates to it
        canonical_index = len(canonical_vertices)
        canonical_vertices.append(vertex)
        
        for idx in duplicate_indices:
            unique_indices[idx] = canonical_index

    # Step 2: Update faces to use only unique vertices
    new_faces = np.array([[unique_indices[vi] for vi in face] for face in faces])

    # Step 3: Create the new unified mesh
    unified_mesh = trimesh.Trimesh(vertices=np.array(canonical_vertices), faces=new_faces)

    return unified_mesh

# Load the OBJ file
mesh = trimesh.load('xyzrgb_dragon.obj')

# Unify vertices within a specified tolerance
unified_mesh = unify_vertices(mesh, tolerance=1e-5)

# Save the unified mesh as an OBJ file
unified_mesh.export('xyzrgb_dragon_fixed.obj')
print("Mesh unified and saved to 'unified_file.obj'")


Mesh unified and saved to 'unified_file.obj'


In [3]:
#mesh = trimesh.load('xyzrgb_dragon.obj')
mesh = trimesh.load('xyzrgb_dragon_fixed.obj')

heat_solver = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
#vector_solver = pp3d.MeshVectorHeatSolver(V, F)

source_vertex = 100

connections = []

for point in [5009, source_vertex]:
    for edge in mesh.edges:
        if edge[0] == point:
            print(edge)
            connections.append(edge[1])
        if edge[1] == point:
            print(edge)
            connections.append(edge[0])

distances = heat_solver.compute_distance(source_vertex)
normalized_distances = (distances - distances.min()) / (distances.max() - distances.min())

#vectors = vector_solver.transport_tangent_vector(1, (1,0))

print(mesh.vertices.shape, distances.shape)#, vectors.shape)

faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

# Add distance values as a scalar field for coloring
#pv_mesh['Distance'] = normalized_distances


# Set up the PyVista plotter
plotter = pv.Plotter()
plotter.add_mesh(pv_mesh, scalars=normalized_distances, cmap='viridis', show_edges=True)

for point in connections:
    vertex_0_position = mesh.vertices[point]
    plotter.add_points(vertex_0_position.reshape(1, 3), color="red", point_size=10)

# Add labels and display
#plotter.add_scalar_bar(title="Distance from Source Vertex")
plotter.show(title="Geodesic Distance Visualization")

plotter.close()

[5006 5009]
[5009 4986]
[4986 5009]
[5009 5003]
[5003 5009]
[5009 5020]
[5007 5009]
[5009 5006]
[5009 5007]
[5024 5009]
[5009 5024]
[5020 5009]
[ 79 100]
[100  82]
[ 82 100]
[100 118]
[100  79]
[1963  100]
[ 100 1963]
[118 100]
(124943, 3) (124943,)


Widget(value='<iframe src="http://localhost:59721/index.html?ui=P_0x1726e1436d0_0&reconnect=auto" class="pyvis…

In [6]:
plotter.deep_clean()